In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
def load_csv_data(csv_path):
    # Load CSV data using pandas
    df = pd.read_csv(csv_path)

    # Assuming the last column is the label and the rest are pixel values
    x_train = df.iloc[:, 1:].values
    y_train = df.iloc[:, 1].values

    # Normalize pixel values
    x_train = x_train / 255.0

    return x_train, y_train

In [3]:
def retrieve_incorrect_predictions(folder_path):
    incorrect_images = []
    correct_labels = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            image = np.array(Image.open(img_path))
            incorrect_images.append(image)

            # Parse actual and predicted labels from filename
            parts = filename.split('_')
            actual_label = int(parts[0])
            predicted_label = int(parts[2])  # Adjust index for predicted label
            correct_labels.append(actual_label)

    return np.array(incorrect_images), np.array(correct_labels)

In [12]:
def preprocess_images(images):
    # Reshape and normalize the images
    processed_images = images.reshape(1,784).astype('float32') / 255.0
    return processed_images


In [16]:
def fine_tune_model(model, x_train_extended, y_train_extended, epochs=5):
    # Fine-tune the model using the extended training set
    model.fit(x_train_extended.reshape(-1,28,28,1), y_train_extended, epochs=epochs)

    return model

In [17]:
def save_fine_tuned_model(model, save_path='fine_tuned_model.h5'):
    model.save(save_path)
    print(f'Fine-tuned model saved at {save_path}')

In [18]:
def main():
    # Load the pre-trained model
    model = models.load_model('cnn_model.h5')

    # Load the original training set from CSV
    csv_path = 'train.csv'
    x_train, y_train = load_csv_data(csv_path)

    # Retrieve incorrect predictions
    incorrect_folder = 'incorrect_predictions'
    incorrect_images, correct_labels = retrieve_incorrect_predictions(incorrect_folder)


    if len(incorrect_images) > 0:
        # Append incorrect predictions to the original training set
        incorrect_images_processed = preprocess_images(incorrect_images)
        x_train_extended = np.concatenate([x_train, incorrect_images_processed], axis=0)
        y_train_extended = np.concatenate([y_train, correct_labels], axis=0)

        # Fine-tune the model
        model = fine_tune_model(model, x_train_extended, y_train_extended)

        # Save the fine-tuned model
        save_fine_tuned_model(model)

if __name__ == "__main__":
    main()


Epoch 1/5
1313/1313 [==============================] - 17s 13ms/step - loss: 0.0817 - accuracy: 0.9956
Epoch 2/5
1313/1313 [==============================] - 18s 14ms/step - loss: 2.3078e-07 - accuracy: 1.0000
Epoch 3/5
1313/1313 [==============================] - 19s 14ms/step - loss: 9.2815e-08 - accuracy: 1.0000
Epoch 4/5
1313/1313 [==============================] - 18s 14ms/step - loss: 4.0249e-08 - accuracy: 1.0000
Epoch 5/5
1313/1313 [==============================] - 18s 13ms/step - loss: 1.8455e-08 - accuracy: 1.0000
Fine-tuned model saved at fine_tuned_model.h5


C:\Users\DELL\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
